# Salmonella sequence preparation
In this notebook we collect all the steps needed to create the results for our new article: *Comparative genome analysis of Hungarian and global strains of Salmonella Infantis*
## Download
We work with two kind of genomes. The first one is complete, we need to download them from NCBI's nuccore. The second ones are raw sequence files from SRA.
### SRA
We download raw sequences from SRA and apply Spades (version 3.14) to assemble it. We created every subdirectories using the Run ids (you can see it in the Supplementary Table 1.). The following script download all the necessary files and perform the assembly.

In [ ]:
%%bash
cd ../denovo
mkdir DRR022727 DRR022732 ERR1014112 ERR1014114 ERR1014116 SRR2939562 SRR3137263 SRR4025935 SRR8100706 SRR8100830 SRR9200029
mkdir DRR022719 DRR022730 DRR022753  ERR1014113 ERR1014115 SRR2587646 SRR2939570 SRR3453168 SRR4025936 SRR8100824 SRR8100840
mkdir DRS016504 DRS016490 DRS016493  DRS016501  DRS018216  ERR1014109 ERR1014118 ERR1014117 ERR1014119 ERR1014111 SRR4098711
mkdir SRR3457701 SRR3457702 SRR4098712 SRR3457703
for i in ERR* SRR* DRR*
do
  cd $i
  fastq-dump --split-3 $i # download raw sequences from SRA
  spades.py --isolate -t 4 -1 *_1.fastq -2 *_2.fastq -o . # apply de-novo assembly
  cd ..
done

Every directory contains a scaffolds.fasta file. We manually copy to folder 'sequences', and rename it.
### Nuccore
From nuccore, we manually download all the necessary files using the web browser. All the sequences was renamed and put into the 'sequences' folder in FASTA format. If the genome contains multiple contigs, all the contigs can be found in the same file.

## Annotate
To annotate all the sequences in the similar way, we use Prokka.

In [ ]:
%%bash
cd ../sequences
for i in *.fasta
do
   /usr/local/molbio/src/prokka/bin/prokka --prefix "${i%.fasta}" --outdir ../annotations/"${i%.fasta}" --genus Salmonella --species enterica --cpus 10 "$i" >>../log 2>>../err
done

## Generating Table 2
Table 2 contains information about filament genes. The following code creates input sequence for Blast.

First, we rename all the contigs. Now every contig name contains the strain name too.

In [5]:
import glob

files = glob.glob("../sequences/*.fasta")
out = open("../blastdb/seq.fasta", "w")
for f in files:
    inp = open(f)
    seqname = f.replace("../sequences/", "").replace(".fasta", "").replace(" ", "_").replace("\\","")
    contig = 1
    for i in inp:
        if i.startswith(">"):
            out.write(">" + seqname + "_" + str(contig) + "\n")
            contig += 1
        else:
            out.write(i)
    inp.close()
out.close()

Create blast database.

In [6]:
%%bash
cd ../blastdb
makeblastdb -in seq.fasta -out salmonella -dbtype nucl



Building a new DB, current time: 01/31/2020 09:14:29
New DB name:   /data/projects/salmonella/blastdb/salmonella
New DB title:  seq.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /data/projects/salmonella/blastdb/salmonella
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 19965 sequences in 4.41693 seconds.


Run blast.

In [7]:
%%bash
cd ../blastdb
tblastn -query query.fasta -db salmonella -out out.tsv -outfmt 7

Filter results. We accept only full length hits with similarity greater than 70%.

In [5]:
seqlengths = {"fliA":239, "fliB":401, "fliC":495, "fliD":467, "fliS":135, "fljA":179, "fljB":506, "hin":190} # sequence length by infoseq
header = {"fliA":0, "fliB":1, "fliC":2, "fliD":3, "fliS":4, "fljA":5, "fljB":6, "hin":7}
outtable = dict()
coverage = dict()
blastout = open("../blastdb/out.tsv")
for i in blastout:
    if i.startswith("#"):
        continue
    fields = i.rstrip().split()
    qseq = fields[0]
    hit  = "_".join(fields[1].split("_")[:-1]) #the last field is just the contig number, we do not need it
    similarity = float(fields[2])
    start = int(fields[6]) - 1
    end   = int(fields[7])
    if similarity > 70.0: # we set similarity here
        if hit not in outtable:
            outtable[hit] = ['-'] * len(header)
            coverage[hit] = dict()
        if qseq not in coverage[hit]:
            coverage[hit][qseq] = [0] * seqlengths[qseq]
        for pos in range(start, end):
            coverage[hit][qseq][pos] = 1
blastout.close()

out = open("../tables/flagellin70.csv", "w")
out.write("strain\tfliA\tfliB\tfliC\tfliD\tfliS\tfljA\tfljB\thin\n")
for row in outtable:
    for col in header:
        if row not in coverage or col not in coverage[row]:
            continue
        s = sum(coverage[row][col])
        if s >= seqlengths[col]:
            outtable[row][header[col]] = '+'
    out.write(row + "\t" + "\t".join(outtable[row]) + "\n")
out.close()

## Tree generation
To generate trees, we use progressiveMauve in a HPC cluster.
### Whole genome of all Salmonella strains

In [ ]:
%%bash
#!/bin/bash

# All Salmonella strains whole genome multiple alignment

#SBATCH -A rbgma
#SBATCH --job-name=infwgs
#SBATCH --time=168:00:00
#SBATCH -n 8

cd /big/scratch/pez5mnr/salmonella/
./progressiveMauve --output-guide-tree=salmonella.wgs.guide_tree --output=salmonella.wgs *.fasta >log.txt

### Core genes of Infantis genes
First, we need to select only Infantis. We created a metadata table 'samples.tsv' and stored it in folder called 'tables'. In that table the second column is 1, if the sample is Infantis and 0 otherwise.

In [3]:
%%bash
cd ../tables
IFS=$(echo -en "\n\b")
for i in `tail -n +2 samples.tsv | awk -F"\t" '{if($2 == 1)print $1}'`
do
  cp ../annotations/"$i"/"$i.gff" ../coregenes_infantis/
done

Next, we identify core and cloud genes using Roary (version). Unfortunaltely Roary does not like space and '\' in the file name, so we need to remove them. The last one was removed manually.

In [ ]:
%%bash
cd ../coregenes_infantis/
# unfortunately roary does not like space in file names
for i in *.gff
do
  mv "$i" ${i/ /_}
done

roary -e --mafft -p 20 *.gff >log 2>err

Extract core genes.

In [ ]:
%%bash
cd ../tables
tail -n +2 samples.tsv | sed 's/ /_/g' | sed 's/\\/_/g' >../coregenes_infantis/tmp2

In [2]:
def revcomp(seq):
    tr = str.maketrans('ATGC', 'TACG')
    return(seq[::-1].translate(tr))

infantis = set()
for i in open("../coregenes_infantis/tmp2"):
    f = i.rstrip().split("\t")
    if f[1] == "1":
        infantis.add(f[0])

gene = open("../coregenes_infantis/gene_presence_absence.csv")
header = gene.readline().replace('"', '').rstrip().split(",")
columns = list()
genes = dict()
for i in range(14,len(header)):
    strain = header[i]
    if strain in infantis:
        columns.append(i)
        genes[strain] = set()

for i in gene:
    f = i.rstrip().split(",")
    for c in columns:
        if f[c] == '""':
            break
    else:
        # We found something
        for c in columns:
            genes[header[c]].add(f[c].replace('"', ''))
            
for strain in genes:
    gff = open("../coregenes_infantis/" + strain + ".gff")
    outseq = open("../coregenes_infantis/" + strain + ".fasta", "w")
    gffmode = True
    genebound = dict()
    fasta = dict()
    for i in gff:
        if i.startswith("#"):
            if i.startswith("##FASTA"):
                gffmode = False
            continue
        if gffmode:
            fields = i.rstrip().split("\t")
            geneid = fields[8].split(";")[0].replace("ID=", "")
            if geneid in genes[strain]:
                if fields[0] not in genebound:
                    genebound[fields[0]] = list()
                genebound[fields[0]].append((geneid, fields[3], fields[4], fields[6])) # I am not sure I need strand information
        else:
            if i.startswith(">"):
                idx = i.rstrip()[1:]
                fasta[idx] = list()
            else:
                fasta[idx].append(i.rstrip())
    gff.close()
    
    for contig in genebound:
        seq = "".join(fasta[contig])
        for entry in genebound[contig]:
            outseq.write(">" + entry[0] + "\n")
            geneseq = seq[int(entry[1])-1:int(entry[2])]
            if entry[3] == "-":
                geneseq = revcomp(geneseq)
            outseq.write(geneseq + "\n")
    outseq.close()

Fasta files were renamed according to the original specification.

Now we can run the multiple alignment:

In [ ]:
%%bash
#!/bin/bash

# Salmonella infantis core genes multiple alignment

#SBATCH -A rbgma
#SBATCH --job-name=infcor
#SBATCH --time=168:00:00
#SBATCH -n 8

cd /big/scratch/pez5mnr/salmonella/
./progressiveMauve --output-guide-tree=infantis.core.guide_tree --output=infantis.core *.fasta >infantis.core.log

### Core genes of all strains

In [5]:
%%bash
cd ../tables
IFS=$(echo -en "\n\b")
for i in `tail -n +2 samples.tsv | awk -F"\t" '{if($2 == 0)print $1}'`
do
  cp ../annotations/"$i"/"$i.gff" ../coregenes_allsamples/
done

In [7]:
%%bash
cd ../coregenes_allsamples/
cp ../coregenes_infantis/*.gff ./
mv S.\ Dublin_CT_02021853.gff S.Dublin_CT_02021853.gff
mv S.Gallinarium\\Pollorum_RKS5078.gff S.Gallinarium_Pollorum_RKS5078.gff
roary -e --mafft -p 10 *.gff >log 2>err

In [3]:
gene = open("../coregenes_allsamples/gene_presence_absence.csv")
header = gene.readline().replace('"', '').rstrip().split(",")
columns = list()
genes = dict()
for i in range(14,len(header)):
    strain = header[i]
    columns.append(i)
    genes[strain] = set()

for i in gene:
    f = i.rstrip().split(",")
    for c in columns:
        if f[c] == '""':
            break
    else:
        # We found something
        for c in columns:
            genes[header[c]].add(f[c].replace('"', ''))
            
for strain in genes:
    gff = open("../coregenes_allsamples/" + strain + ".gff")
    outseq = open("../coregenes_allsamples/" + strain + ".fasta", "w")
    gffmode = True
    genebound = dict()
    fasta = dict()
    for i in gff:
        if i.startswith("#"):
            if i.startswith("##FASTA"):
                gffmode = False
            continue
        if gffmode:
            fields = i.rstrip().split("\t")
            geneid = fields[8].split(";")[0].replace("ID=", "")
            if geneid in genes[strain]:
                if fields[0] not in genebound:
                    genebound[fields[0]] = list()
                genebound[fields[0]].append((geneid, fields[3], fields[4], fields[6])) # I am not sure I need strand information
        else:
            if i.startswith(">"):
                idx = i.rstrip()[1:]
                fasta[idx] = list()
            else:
                fasta[idx].append(i.rstrip())
    gff.close()
    
    for contig in genebound:
        seq = "".join(fasta[contig])
        for entry in genebound[contig]:
            outseq.write(">" + entry[0] + "\n")
            geneseq = seq[int(entry[1])-1:int(entry[2])]
            if entry[3] == "-":
                geneseq = revcomp(geneseq)
            outseq.write(geneseq + "\n")
    outseq.close()

In [ ]:
%%bash
cd ../coregenes_allsamples

mv S.Dublin_CT_02021853.fasta "S. Dublin_CT_02021853.fasta"
mv S.Gallinarium_Pollorum_RKS5078.fasta "S.Gallinarium\Pollorum_RKS5078.fasta"

./progressiveMauve --output-guide-tree=allsamples.core.guide_tree --output=allsamples.core *.fasta >allsamples.core.log

### Cloude genes of Infantis

In [4]:
gene = open("../coregenes_infantis/gene_presence_absence.csv")
header = gene.readline().replace('"', '').rstrip().split(",")
columns = list()
genes = dict()
for i in range(14,len(header)):
    strain = header[i]
    columns.append(i)
    genes[strain] = set()

for i in gene:
    f = i.rstrip().split(",")
    found_in_strains = 0
    for c in columns:
        if f[c] != '""':
            found_in_strains += 1
    percentage = float(found_in_strains) / float(len(columns))
    if percentage < 0.15:
        # We found something
        for c in columns:
            if f[c] != '""':
                genes[header[c]].add(f[c].replace('"', ''))
            
for strain in genes:
    gff = open("../coregenes_infantis/" + strain + ".gff")
    outseq = open("../cloudgenes_infantis/" + strain + ".fasta", "w")
    gffmode = True
    genebound = dict()
    fasta = dict()
    for i in gff:
        if i.startswith("#"):
            if i.startswith("##FASTA"):
                gffmode = False
            continue
        if gffmode:
            fields = i.rstrip().split("\t")
            geneid = fields[8].split(";")[0].replace("ID=", "")
            if geneid in genes[strain]:
                if fields[0] not in genebound:
                    genebound[fields[0]] = list()
                genebound[fields[0]].append((geneid, fields[3], fields[4], fields[6])) # I am not sure I need strand information
        else:
            if i.startswith(">"):
                idx = i.rstrip()[1:]
                fasta[idx] = list()
            else:
                fasta[idx].append(i.rstrip())
    gff.close()
    
    for contig in genebound:
        seq = "".join(fasta[contig])
        for entry in genebound[contig]:
            outseq.write(">" + entry[0] + "\n")
            geneseq = seq[int(entry[1])-1:int(entry[2])]
            if entry[3] == "-":
                geneseq = revcomp(geneseq)
            outseq.write(geneseq + "\n")
    outseq.close()

In [ ]:
%%bash
#!/bin/bash

# Salmonella infantis cloud genes multiple alignment

#SBATCH -A rbgma
#SBATCH --job-name=infcloud
#SBATCH --time=168:00:00
#SBATCH -n 8

cd /big/scratch/pez5mnr/salmonella/
./progressiveMauve --output-guide-tree=infantis.cloud.guide_tree --output=infantis.cloud *.fasta >infantis.cloud.log

### Cloud genes of all strains

In [5]:
gene = open("../coregenes_allsamples/gene_presence_absence.csv")
header = gene.readline().replace('"', '').rstrip().split(",")
columns = list()
genes = dict()
for i in range(14,len(header)):
    strain = header[i]
    columns.append(i)
    genes[strain] = set()

for i in gene:
    f = i.rstrip().split(",")
    found_in_strains = 0
    for c in columns:
        if f[c] != '""':
            found_in_strains += 1
    percentage = float(found_in_strains) / float(len(columns))
    if percentage < 0.15:
        # We found something
        for c in columns:
            if f[c] != '""':
                genes[header[c]].add(f[c].replace('"', ''))
            
for strain in genes:
    gff = open("../coregenes_allsamples/" + strain + ".gff")
    outseq = open("../cloudgenes_allsamples/" + strain + ".fasta", "w")
    gffmode = True
    genebound = dict()
    fasta = dict()
    for i in gff:
        if i.startswith("#"):
            if i.startswith("##FASTA"):
                gffmode = False
            continue
        if gffmode:
            fields = i.rstrip().split("\t")
            geneid = fields[8].split(";")[0].replace("ID=", "")
            if geneid in genes[strain]:
                if fields[0] not in genebound:
                    genebound[fields[0]] = list()
                genebound[fields[0]].append((geneid, fields[3], fields[4], fields[6])) # I am not sure I need strand information
        else:
            if i.startswith(">"):
                idx = i.rstrip()[1:]
                fasta[idx] = list()
            else:
                fasta[idx].append(i.rstrip())
    gff.close()
    
    for contig in genebound:
        seq = "".join(fasta[contig])
        for entry in genebound[contig]:
            outseq.write(">" + entry[0] + "\n")
            geneseq = seq[int(entry[1])-1:int(entry[2])]
            if entry[3] == "-":
                geneseq = revcomp(geneseq)
            outseq.write(geneseq + "\n")
    outseq.close()

This tree generation run on different machine, because I need to hurry.

In [ ]:
%%bash
cd ../cloudgenes_allsamples/
../coregenes_allsamples/progressiveMauve --output-guide-tree=salmonella.cloud.guide_tree --output=salmonella.cloud *.fasta >salmonella.cloud.log

### Genomic islands
To predict genomic islands (GI), we use IslandViewer 4. We upload UK-1973 1326/28 genomes and predict GIs. We downloaded the results in GeneBank format and tab separated table. We extract the sequences using seqret.

In [3]:
%%bash
cd ../genomicisland/
tail -n +2 UK1973132628.tsv | cut -f 1,2 | sort -u | awk '{print "seqret -sbegin "$1" -send "$2" UK1973132628.gbk -stdout -auto -sid gi."NR}' >extract.sh
chmod a+x extract.sh
./extract.sh >GI.fasta

We use the previously created Blast database to find the selected GIs in other strains.

In [4]:
%%bash
cd ../blastdb
blastn -query ../genomicisland/GI.fasta -db salmonella -outfmt 7 -out gi.blast.out

The following code snippets choose the right coordinates to later extract the sequences.

In [2]:
blastout  = open("../blastdb/gi.blast.out")
gi_length = dict()
coverage  = dict()
for i in blastout:
    if i.startswith("#"):
        continue
    fields = i.rstrip().split("\t")
    gi_id  = fields[0]
    strain = "_".join(fields[1].split("_")[:-1])
    perc   = float(fields[2])
    alnlen = int(fields[3])
    if perc < 0.9: # HSP similarity should be larger than 90%
        continue
    if alnlen < 200: # if the HSP too small, we drop it
        continue
    if gi_id not in gi_length:
        gi_length[gi_id] = alnlen # the first hit will be the same, so the alignment length gives us the length of the GI
        coverage[gi_id] = dict()
    if strain not in coverage[gi_id]:
        coverage[gi_id][strain] = list()
    coverage[gi_id][strain].append([int(fields[6]), int(fields[7]), int(fields[8]), int(fields[9]), fields[1]])
blastout.close()

for gi_id in coverage:
    l = gi_length[gi_id]
    for strain in coverage[gi_id]:
        cov = [0] * l
        straincoord = list() # genomic island coordinates in the specified strain
        for hsp in coverage[gi_id][strain]:
            gi_start = hsp[0] -1
            gi_end   = hsp[1]
            for i in range(gi_start, gi_end):
                cov[i] = 1
        if (l * 0.9) < float(sum(cov)):
            # strain accepted, because the HSPs cover more than 90%
            # some part of the genomic island is covered more than one
            # we select the larger parts
            removeindex = set()
            for i in range(len(coverage[gi_id][strain]) - 1):
                for j in range(i + 1, len(coverage[gi_id][strain])):
                    hsp1 = coverage[gi_id][strain][i]
                    hsp2 = coverage[gi_id][strain][j]
                    if hsp2[0] > hsp1[0] and hsp2[1] < hsp1[1]:
                        # hsp2 is inside hsp1
                        removeindex.add(j)
                    if hsp1[0] > hsp2[0] and hsp1[1] < hsp2[1]:
                        removeindex.add(i)
            removeindex = sorted(removeindex, reverse = True) # deleting from an array will reindex it
            for i in removeindex:
                del coverage[gi_id][strain][i]

In [9]:
def extract(seq, poslist, strain):
    seq = "".join(seq)
    outname = "_".join(strain.split("_")[:-1])
    out = open("../genomicisland/ukislands/" + outname + ".fasta", "a")
    for p in poslist:
        start = p[0] - 1 #strings are zero based
        end = p[1]
        out.write(">" + strain + "\n")
        out.write(seq[start:end] + "\n")
    out.close()

positions = dict()
for gi_id in coverage:
    for strain in coverage[gi_id]:
        for contig in coverage[gi_id][strain]:
            contig_name = contig[4]
            contig_start = min(contig[2], contig[3])
            contig_end = max(contig[2], contig[3])
            if contig_name not in positions:
                positions[contig_name] = list()
            positions[contig_name].append([contig_start, contig_end])

fasta = open("../blastdb/seq.fasta")
seq = list()
for i in fasta:
    if i.startswith('>'):
        if len(seq) > 0 and seqname in positions:
            extract(seq, positions[seqname], seqname)
        seqname = i.rstrip()[1:]
        seq = list()
    else:
        seq.append(i.rstrip())
fasta.close()
if len(seq) > 0 and seqname in positions:
    extract(seq, positions[seqname], seqname)

Rename samples to represents correctly our strains.

In [10]:
%%bash
cd ../tables

awk -F"\t" '{if(NR>1)print $1}' samples.tsv >a
ls ../genomicisland/ukislands/ >b
paste b a | awk -F"\t" '{print "mv "$1" \""$2".fasta\""}' >../genomicisland/ukislands/cmd.sh
rm a b

cd ../genomicisland/ukislands/

for i in *.fasta
do
  fold -w 60 $i >tmp
  mv tmp $i
done

chmod a+x cmd.sh
./cmd.sh

mv: ‘Mexico-2008-1.fasta’ and ‘Mexico-2008-1.fasta’ are the same file
mv: ‘Mexico-2008-2.fasta’ and ‘Mexico-2008-2.fasta’ are the same file
mv: ‘Mexico-2008-3.fasta’ and ‘Mexico-2008-3.fasta’ are the same file
mv: ‘Mexico-2008-4.fasta’ and ‘Mexico-2008-4.fasta’ are the same file
mv: ‘Mexico-2008-5.fasta’ and ‘Mexico-2008-5.fasta’ are the same file
mv: ‘Mexico-2008-CFSAN047352.fasta’ and ‘Mexico-2008-CFSAN047352.fasta’ are the same file
mv: ‘Mexico-2009-CFSAN047411.fasta’ and ‘Mexico-2009-CFSAN047411.fasta’ are the same file
mv: ‘Mexico-2009-CFSAN047417.fasta’ and ‘Mexico-2009-CFSAN047417.fasta’ are the same file
mv: ‘Mexico-2010-CFSAN047496.fasta’ and ‘Mexico-2010-CFSAN047496.fasta’ are the same file
mv: ‘Mexico-2010-CFSAN047510.fasta’ and ‘Mexico-2010-CFSAN047510.fasta’ are the same file
mv: ‘S.Agona_460004.fasta’ and ‘S.Agona_460004.fasta’ are the same file
mv: ‘S.Agona_SL483.fasta’ and ‘S.Agona_SL483.fasta’ are the same file
mv: ‘SAl147[2011].fasta’ and ‘SAl147[2011].fasta’ are the 

In [11]:
%%bash
cd ../genomicisland/ukislands/
mkdir infantis
mv Brasil-201* Nigeria-2009\ BCW_2699.fasta Peru-2016\ SPE101.fasta SAl147\[2011\].fasta Sal280\[2012\].fasta Senegal\ SARB27.fasta SI0* infantis/
mv Hungary-* Israel-* Italy-20* Japan-* Mexico-20* Switzerland-201* UK-1973\ 1326\\28.fasta USA-201* infantis/
cd infantis
../../../coregenes_allsamples/progressiveMauve --output-guide-tree=infantis.gi.guide_tree --output=infantis.gi *.fasta >infantis.gi.log